In [1]:
from transformers import AutoTokenizer

#加载分词工具
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer

/Users/kennywu/opt/anaconda3/envs/dl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

AutoTokenizer

In [3]:
from datasets import load_dataset
from datasets import load_from_disk

#加载数据集
#从网络加载
#datasets = load_dataset(path='glue', name='sst2')

#从本地磁盘加载数据
datasets = load_from_disk('./data/glue_sst2')


#分词
def f(data):
    return tokenizer(
        data['sentence'],
        padding='max_length',
        truncation=True,
        max_length=30,
    )


datasets = datasets.map(f, batched=True, batch_size=1000, num_proc=4)

#取数据子集，否则数据太多跑不动
dataset_train = datasets['train'].shuffle().select(range(1000))
dataset_test = datasets['validation'].shuffle().select(range(200))

del datasets

dataset_train

Loading cached processed dataset at /Users/kennywu/Documents/Github_repos/Huggingface_CN_Toturials/data/glue_sst2/train/cache-d5a332e72039e5e9_*_of_00004.arrow
Loading cached processed dataset at /Users/kennywu/Documents/Github_repos/Huggingface_CN_Toturials/data/glue_sst2/validation/cache-54ebd1d2495e5e86_*_of_00004.arrow
Loading cached processed dataset at /Users/kennywu/Documents/Github_repos/Huggingface_CN_Toturials/data/glue_sst2/test/cache-47c71c904c75dba8_*_of_00004.arrow


Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

用AutoModelForSequenceClassification 

In [5]:
from transformers import AutoModelForSequenceClassification

#加载模型 AutoModelForSequenceClassification，指定为2分类
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased',
                                                           num_labels=2)

print(sum([i.nelement() for i in model.parameters()]) / 10000)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

10831.181


In [6]:
import numpy as np
from datasets import load_metric
from transformers.trainer_utils import EvalPrediction

#加载评价函数
#有时会因为网络问题卡主,反复尝试会成功的
metric = load_metric('accuracy')


#定义评价函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits.argmax(axis=1)
    return metric.compute(predictions=logits, references=labels)


#模拟测试输出
eval_pred = EvalPrediction(
    predictions=np.array([[0, 1], [2, 3], [4, 5], [6, 7]]),
    label_ids=np.array([1, 1, 1, 1]),
)

compute_metrics(eval_pred)

/var/folders/w9/g6sxmrzn3bj5clmc2jx9vzt40000gn/T/ipykernel_53185/623941755.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


{'accuracy': 1.0}

# transformers的 TrainingArguments, Trainer
transformers库中用于训练模型的两个重要类。

## TrainingArguments是一个用于设置训练参数的类，
包含了许多用于控制训练过程的参数，例如训练时的学习率、batch size、epoch数、优化器类型等等。这些参数可以通过实例化TrainingArguments类并传递相应的参数来设置。

## Trainer是一个用于训练模型的类，
它可以接受一个模型和一个数据集，并使用指定的训练参数进行训练。
在训练过程中，Trainer类可以自动进行数据加载、优化器设置、模型评估、日志记录等操作。Trainer类还可以使用多个GPU进行分布式训练。

核心：可以避免手动编写训练循环（training loop）的代码，不需要再写def train函数了。Trainer类已经实现了完整的训练过程，包括数据加载、前向传播、反向传播、梯度更新、模型评估等步骤。

In [21]:
from transformers import TrainingArguments, Trainer

#初始化训练参数
args = TrainingArguments(output_dir='./output_dir', # 模型参数保存地址
                         evaluation_strategy='epoch',
                         no_cuda=True)
# 些参数可以通过实例化TrainingArguments类并传递相应的参数来设置。
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.weight_decay = 1e-2
args.per_device_eval_batch_size = 32
args.per_device_train_batch_size = 16

#初始化训练器： 在这里定义
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train, # Trainer的好处，连dataset都不用定义了！
    eval_dataset=dataset_test, # eval data也有好处
    compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:

# 不训练，直接测试初始模型
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32
100%|██████████| 7/7 [00:11<00:00,  1.62s/it]


{'eval_loss': 0.4577715992927551,
 'eval_accuracy': 0.81,
 'eval_runtime': 13.5759,
 'eval_samples_per_second': 14.732,
 'eval_steps_per_second': 0.516}

可以避免手动编写训练循环（training loop）的代码，不需要再写def train函数了。Trainer类已经实现了完整的训练过程，包括数据加载、前向传播、反向传播、梯度更新、模型评估等步骤。

In [8]:
#训练 1个epoch
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/kennywu/opt/anaconda3/envs/dl/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 63
  Number of trainable parameters = 108311810
100%|██████████| 63/63 [04:41<00:00,  3.88s/it]The following columns in the evaluation set don't h

{'eval_loss': 0.4577715992927551, 'eval_accuracy': 0.81, 'eval_runtime': 14.0756, 'eval_samples_per_second': 14.209, 'eval_steps_per_second': 0.497, 'epoch': 1.0}
{'train_runtime': 295.1052, 'train_samples_per_second': 3.389, 'train_steps_per_second': 0.213, 'train_loss': 0.629571278889974, 'epoch': 1.0}


TrainOutput(global_step=63, training_loss=0.629571278889974, metrics={'train_runtime': 295.1052, 'train_samples_per_second': 3.389, 'train_steps_per_second': 0.213, 'train_loss': 0.629571278889974, 'epoch': 1.0})

In [11]:
# 再次评价模型
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32
100%|██████████| 7/7 [00:11<00:00,  1.65s/it]


{'eval_loss': 0.4577715992927551,
 'eval_accuracy': 0.81,
 'eval_runtime': 13.7887,
 'eval_samples_per_second': 14.505,
 'eval_steps_per_second': 0.508}

In [12]:
#保存模型的【参数】，不是模型本身
trainer.save_model(output_dir='./output_dir')

Saving model checkpoint to ./output_dir
Configuration saved in ./output_dir/config.json
Model weights saved in ./output_dir/pytorch_model.bin


# 重新加在保存的模型，来预测

In [17]:
import torch


def collate_fn(data):
    label = [i['label'] for i in data]
    input_ids = [i['input_ids'] for i in data]
    token_type_ids = [i['token_type_ids'] for i in data]
    attention_mask = [i['attention_mask'] for i in data]

    label = torch.LongTensor(label)
    input_ids = torch.LongTensor(input_ids)
    token_type_ids = torch.LongTensor(token_type_ids)
    attention_mask = torch.LongTensor(attention_mask)

    return label, input_ids, token_type_ids, attention_mask


#单独 test数据加载器:  因为不是trainer ，没有办法，只能用torch dataset
loader_test = torch.utils.data.DataLoader(dataset=dataset_test, # 还是上面的validation
                                          batch_size=4,
                                          collate_fn=collate_fn,
                                        #   shuffle=True,
                                            shuffle=False,
                                          drop_last=True)

for i, (label, input_ids, token_type_ids,
        attention_mask) in enumerate(loader_test):
    break

# label, input_ids, token_type_ids, attention_mask

In [18]:
import torch


# 测试
def test():
    #加载参数
    model.load_state_dict(torch.load('./output_dir/pytorch_model.bin'))

    model.eval()

    #运算
    out = model(input_ids=input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask)

    #[4, 2] -> [4]
    out = out['logits'].argmax(dim=1)

    correct = (out == label).sum().item()

    return correct / len(label)


test()

1.0